In [1]:
# Jupyter setup to expand cell display to 100% width on your screen (optional)
# Import relevant modules and setup for calling glmnet
%reset -f
%matplotlib inline

from sqlalchemy import create_engine
import sys
import pandas as pd
import numpy as np
import scipy, importlib, pprint, matplotlib.pyplot as plt, warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.impute import KNNImputer

import statsmodels.api as sm
import statsmodels.genmod as genmod
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_absolute_error

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score
import sklearn.model_selection as ms

import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Load TSR and attributes data from DB

In [2]:
localhost = {'user': 'postgres', 'password': 'postgres', 'host': 'localhost', 'port': 5432, 'db': 'fiadb'}
params = 'postgresql://{0}:{1}@{2}:{3}/{4}'
engine = create_engine(params.format(localhost['user'], localhost['password'], localhost['host'], localhost['port'], localhost['db']))
# geom_sql = """select distinct grid_id, grid_geom from fs_fiadb.pergrid"""
pergrid_base = """select distinct * from predictor.pergrid_base"""
pergrid_base_df = pd.read_sql(pergrid_base, engine)

In [3]:
pergrid_base_df.columns

Index(['grid_id', 'aet', 'ai', 'art', 'ewd', 'fa', 'map', 'mat', 'mpdq',
       'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn', 'mfdf', 'alt', 'shg',
       'mtwq', 'wkb_geometry', 'tsr', 'wa', 'ha', 'wkt', 'lat', 'lon'],
      dtype='object')

## Process TSR predictor and outcome varaibles

In [4]:
y = pergrid_base_df['tsr']
grid_id = pergrid_base_df['grid_id']

pred_var= ['aet', 'ai', 'art', 'ewd', 'fa', 'map', 'mat', 'mpdq',
       'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn', 'mfdf', 'alt','shg', 'mtwq', 'wa']

pergrid_base_selected_df = pergrid_base_df[pred_var]

In [5]:
pergrid_base_selected_df.head(2)

,aet,ai,art,ewd,fa,map,mat,mpdq,mtcq,pet,psn,ra,rmap,rmat,tsn,mfdf,alt,shg,mtwq,wa
0,6000.000,0.367,21.192,-2523666.200,7.706,681.000,20.036,87.000,16.299,1826.768,50.412,16.000,2.833,0.604,444.860,2.270,11.598,None,28.793,4.117
1,65535.000,0.383,20.234,-2537902.123,14.150,699.000,20.299,92.000,16.340,1811.928,49.798,14.000,2.000,0.621,444.260,2.100,5.983,5,28.717,11.815


### fill in no-data grid with value from neighboring grids

In [6]:
imputer = KNNImputer(n_neighbors=5)
pergrid_base_selected_filled = imputer.fit_transform(pergrid_base_selected_df)
pergrid_base_df = pd.DataFrame(pergrid_base_selected_filled)

In [7]:
pergrid_base_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,6000.000,0.367,21.192,-2523666.200,7.706,681.000,20.036,87.000,16.299,1826.768,50.412,16.000,2.833,0.604,444.860,2.270,11.598,5.800,28.793,4.117
1,65535.000,0.383,20.234,-2537902.123,14.150,699.000,20.299,92.000,16.340,1811.928,49.798,14.000,2.000,0.621,444.260,2.100,5.983,5.000,28.717,11.815
2,65535.000,1.304,0.164,-524.709,0.071,1148.500,0.310,131.500,21.304,1918.111,0.714,1.000,1.833,0.242,3.699,6.524,0.012,2.000,28.626,302.905
3,5135.000,0.290,23.950,-2941595.415,3.004,565.000,20.194,76.000,15.517,1972.128,43.001,108.000,2.333,0.392,492.281,2.120,61.887,5.400,29.459,1.253
4,5645.000,0.302,23.526,-2959517.411,10.035,580.000,20.710,80.000,15.754,1928.969,44.328,34.000,3.417,0.621,489.587,2.120,32.271,5.000,29.300,4.248


In [8]:
pergrid_base_df.columns = pred_var

In [9]:
from sklearn.preprocessing import StandardScaler

# ss = MinMaxScaler()
ss = StandardScaler()
X_std = ss.fit_transform(pergrid_base_df)

Xstd=pd.DataFrame(data=X_std[0:,0:],
                index=pergrid_base_df.index,
                columns=pred_var)

In [10]:
Xstd.head()

,aet,ai,art,ewd,fa,map,mat,mpdq,mtcq,pet,psn,ra,rmap,rmat,tsn,mfdf,alt,shg,mtwq,wa
0,-0.182,-0.752,-0.957,-1.362,-1.117,-0.447,2.494,-0.566,2.309,0.906,1.540,-0.823,-0.516,-0.586,-1.039,-2.341,-0.934,0.658,1.569,-0.242
1,4.273,-0.712,-1.103,-1.379,-1.057,-0.404,2.551,-0.511,2.315,0.868,1.500,-0.827,-0.570,-0.578,-1.042,-2.369,-0.941,0.236,1.553,-0.058
2,4.273,1.657,-4.172,1.569,-1.188,0.670,-1.852,-0.080,3.060,1.136,-1.705,-0.854,-0.581,-0.752,-3.356,-1.620,-0.949,-1.344,1.532,6.922
3,-0.246,-0.952,-0.535,-1.848,-1.161,-0.724,2.528,-0.686,2.191,1.272,1.056,-0.630,-0.549,-0.683,-0.790,-2.366,-0.868,0.447,1.718,-0.311
4,-0.208,-0.921,-0.600,-1.869,-1.095,-0.688,2.642,-0.642,2.227,1.163,1.143,-0.785,-0.478,-0.578,-0.804,-2.366,-0.907,0.236,1.683,-0.239


In [11]:
cleaned_df = pd.concat([pergrid_base_df, y], axis=1, sort=False)
cleaned_df.to_csv('/Users/lianfeng/Document/PhD/writings/first_maualscript/tsrmodel/cleaned_pergrid.csv')

In [12]:
pergrid_base_df = Xstd
pergrid_base_df = pergrid_base_df.assign(intercept = 1.)

In [13]:
import statistics 

### check error distribution to see if using Possion or NegativeBinomial

In [14]:
poisson_model = sm.GLM(y, pergrid_base_df,family=sm.families.Poisson())
poisson_results = poisson_model.fit()
y_pred= poisson_model.predict(params=poisson_results.params, exposure=None, offset=None, linear=False)
poisson_residual = y_pred - y

In [15]:
statistics.variance(poisson_residual)

32.73545463936625

In [16]:
statistics.mean(poisson_residual)

1.8718041123899384e-14

## Build GLM Model

In [17]:
# poisson_model = sm.GLM(y, pergrid_base_df,family=sm.families.Poisson())
poisson_model = sm.GLM(y, pergrid_base_df,family=sm.families.NegativeBinomial())

In [18]:
poisson_results = poisson_model.fit()

In [19]:
print(poisson_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    tsr   No. Observations:                15310
Model:                            GLM   Df Residuals:                    15289
Model Family:        NegativeBinomial   Df Model:                           20
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -53706.
Date:                Mon, 01 Mar 2021   Deviance:                       3141.8
Time:                        08:01:14   Pearson chi2:                 2.97e+03
No. Iterations:                    12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
aet           -0.0030      0.011     -0.281      0.7

In [20]:
params_df = poisson_results.params
params_df = params_df.to_frame()

In [21]:
params_df['var'] = params_df.index

In [22]:
params_df.columns = ['coefficent','var']

In [23]:
params_df['coefficent_abs'] = abs(params_df['coefficent'])
params_df.sort_values(by='coefficent_abs', ascending=False)

,coefficent,var,coefficent_abs
intercept,2.448,intercept,2.448
fa,0.394,fa,0.394
map,0.349,map,0.349
rmat,0.337,rmat,0.337
art,0.285,art,0.285
alt,-0.275,alt,0.275
ewd,0.239,ewd,0.239
ai,-0.238,ai,0.238
mpdq,0.236,mpdq,0.236
ra,-0.204,ra,0.204


In [24]:
# # null hypothesis: predictors have no effect; A low p-value (< 0.05) indicates that the null hypothesis can be rejected
# print("P values of predictors: ")
# print(poisson_results.pvalues.sort_values(ascending=True))

In [25]:
print("Predictors which are not statistically significant:")
print(poisson_results.pvalues[poisson_results.pvalues > 0.05])

Predictors which are not statistically significant:
aet    0.779
mat    0.548
mtcq   0.756
pet    0.082
psn    0.439
rmap   0.297
tsn    0.932
shg    0.149
mtwq   0.712
wa     0.094
dtype: float64


### Evaluation

In [26]:
perc_dev_explained_all = (1 - (poisson_results.deviance/poisson_results.null_deviance))*100
print("R-squared (% Deviance explained): {}".format(perc_dev_explained_all))

R-squared (% Deviance explained): 75.14713030739658


In [27]:
sst_val = sum(map(lambda Xstd: np.power(Xstd,2),y-np.mean(y))) 
sse_val = sum(map(lambda Xstd: np.power(Xstd,2),poisson_results.resid_response)) 
r2 = 1.0 - sse_val/sst_val
print(r2)

0.7163992636680263


In [28]:
y_pred= poisson_model.predict(params=poisson_results.params, exposure=None, offset=None, linear=False)
r_squared_val = r2_score(y, y_pred)
print(r_squared_val)

0.7163992636680225


## Build Cross-validation GLM Model

In [29]:
class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels GLM w/ Possion """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
#         self.model_ = self.model_class(y, X, sm.families.Poisson())
        self.model_ = self.model_class(y, X, sm.families.NegativeBinomial())
        self.results_ = self.model_.fit()
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)

In [30]:
wrapped_possion_glm = SMWrapper(sm.GLM)

In [31]:
wrapped_possion_glm.fit(Xstd,y)

In [32]:
r2_cross_val = cross_val_score(wrapped_possion_glm, Xstd, y, scoring='r2', cv = ms.StratifiedKFold(n_splits=10, shuffle = True))
print("Cross-validated R2: ",)
statistics.mean(r2_cross_val.tolist())

Cross-validated R2: 


0.715231324083713

## Evaluate individual predictor variables

In [33]:
# reduced_results_df = pd.DataFrame(columns=['pred_var', 'r_squared', 'perc_dev_explained', 'r2','null_deviance','deviance','abs_coeff'])  

# for var in pred_var:
#     X_reduced = Xstd[var]
#     reduced_model = sm.GLM(y, X_reduced, family=sm.families.NegativeBinomial())
#     reduced_results = reduced_model.fit()

#     y_pred= reduced_model.predict(params=reduced_results.params, exposure=None, offset=None, linear=False)
#     r_squared_val = r2_score(y, y_pred)
#     perc_dev_explained = 1-(reduced_results.deviance/reduced_results.null_deviance)
    
#     sst_val = sum(map(lambda X_reduced: np.power(X_reduced,2),y-np.mean(y))) 
#     sse_val = sum(map(lambda X_reduced: np.power(X_reduced,2),reduced_results.resid_response)) 
#     r2 = 1.0 - sse_val/sst_val

#     deviance = reduced_results.deviance
#     null_deviance = reduced_results.null_deviance
#     abs_coeff_val = abs(reduced_results.params.values[0])
    
#     reduced_results_df = reduced_results_df.append({'pred_var': var,
#                                  'r_squared':r_squared_val,
#                                  'perc_dev_explained': perc_dev_explained,
#                                  'r2': r2,
#                                  'null_deviance':null_deviance,    
#                                  'deviance':deviance,
#                                  'abs_coeff': abs_coeff_val},
#                                 ignore_index=True)

In [34]:
# reduced_results_df

## Model Evaluation

In [35]:
x_train, x_test, y_train, y_test = train_test_split(Xstd, y, test_size=0.2, random_state=12345)

In [36]:
y_test_pred = wrapped_possion_glm.predict(x_test)
print('test MAE', mean_absolute_error(y_test_pred, y_test))
print('test r2', r2_score(y_test, y_test_pred))

test MAE 4.71316773843979
test r2 0.7397538296105763


In [37]:
glm_y_test = pd.DataFrame(
    {'tsr': y_test,
     'tsr_predicted': y_test_pred})
glm_y_test.to_sql(name='glm_y_test', con=engine, schema='predictor', if_exists='replace', index=False)

In [38]:
glm_y_test.shape

(3062, 2)

## Model Prediction

In [39]:
Y_pred = wrapped_possion_glm.predict(Xstd)

In [40]:
pergrid_all_predicted = pd.DataFrame(
    {'grid_id': grid_id,
     'tsr': y,
     'tsr_predicted': Y_pred.tolist()})

In [41]:
pergrid_all_predicted.head()

,grid_id,tsr,tsr_predicted
0,110,4.000,5.736
1,111,2.000,5.790
2,195,5.000,3.713
3,337,1.000,4.629
4,338,1.000,5.201


In [42]:
pergrid_all_predicted.to_sql(name='glm', con=engine, schema='predictor', if_exists='replace', index=False)

In [43]:
update_geom = """
alter table predictor.glm add column if not exists wkb_geometry geometry(Polygon,4269);
update predictor.glm A SET wkb_geometry = B.wkb_geometry
FROM predictor.pergrid_base B
WHERE A.grid_id = B.grid_id
"""

In [44]:
connection = engine.connect()
connection.execute(update_geom)

In [45]:
update_residual = """
alter table predictor.glm add column residual double precision;
update predictor.glm set residual = (tsr_predicted-tsr);
"""

In [46]:
connection = engine.connect()
connection.execute(update_residual)